In [4]:
%pip install -q openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install --upgrade -q langchain-openai


Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install -q pymupdf langchain

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import openai
import warnings
from dotenv import load_dotenv
warnings.filterwarnings("ignore")

In [3]:
load_dotenv()

True

In [8]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [1]:
from typing import List, Optional
import fitz
from pydantic import BaseModel, Field
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser, JsonOutputFunctionsParser

In [14]:
def read_pdf(file_path: str) -> str:
    with fitz.open(file_path) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
    return text

In [15]:
model = ChatOpenAI(model = "gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

In [16]:
# Define Pydantic models for the data we want to extract
class Interaction(BaseModel):
    """Information about molecular interactions mentioned."""
    entities_involved: List[str]
    interaction_type: str
    interaction_details: Optional[str]

class Molecular_Interactions(BaseModel):
    """Information to extract."""
    interactions: List[Interaction]

In [17]:
paper_extraction_function = [
    convert_pydantic_to_openai_function(Molecular_Interactions)
]
extraction_model = model.bind(
    functions=paper_extraction_function,
    function_call={"name":"Molecular_Interactions"}
)

/opt/homebrew/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [18]:
# Write a prompt for extracting molecular interactions
template = """A scientific paper will be provided to you. Ensure to extract all molecular interactions mentioned in this paper.

Identify the entities involved in each interaction and describe the type of interaction (e.g., binding, inhibition, activation).

If detailed interaction mechanisms or effects are mentioned, include those as well.

Do not include any background information or general knowledge not directly related to the interactions described in the paper.

Only extract and report on the specific molecular interactions that the paper presents."""


In [19]:
# Define the extraction chain
prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])

extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="interactions")

In [20]:
doc = read_pdf("/Users/favourjames/Downloads/gsoc_llm/biology-12-01334.pdf")

In [21]:
page_content = doc[1481:10000]
print(page_content)

Simple Summary: Viral outbreaks continue to be an obstacle to human health, with the Severe
Acute Respiratory Syndrome-related Coronavirus (SARS-CoV-2) pandemic shedding light on the
current vulnerabilities of our healthcare system. Understanding viral pathology is paramount in
developing methods to combat infection. The study of microRNAs (miRNAs) is relatively new to the
world of virology research. These small strands of nucleotides are a versatile tool in the regulation of
gene expression. Various miRNAs have roles in immune development, immune and inﬂammatory
responses, and viral infections. Changes in miRNA expression can be a double-edged sword, used
to alter cell activities to help the host ﬁght infections or taken advantage of by viruses to enhance
infection. Uncovering these interactions and their implications can provide direction for therapeutic
and diagnostic advancement. Further, miRNAs may have the potential to predict the severity of
viral infection and possible health o

In [22]:
results = extraction_chain.invoke({"input": page_content})

In [23]:
results

[{'entities_involved': ['miRNA', 'mRNA'],
  'interaction_type': 'regulation',
  'interaction_details': "miRNA regulates mRNA translation by binding to the 3' UTR, coding sequence (CDS), or 5' UTR of mRNA to affect stability or translation."},
 {'entities_involved': ['miRNA', 'mRNA'],
  'interaction_type': 'regulation',
  'interaction_details': "miRNA can trigger mRNA degradation pathway to downregulate protein production by binding to mRNA with a short complimentary sequence in the 3' UTR."},
 {'entities_involved': ['miRNA', 'mRNA'],
  'interaction_type': 'regulation',
  'interaction_details': 'miRNA may upregulate translation in some cases by directly interacting with mRNA.'},
 {'entities_involved': ['miRNA', 'mRNA'],
  'interaction_type': 'regulation',
  'interaction_details': 'In rare cases, miRNA can cleave mRNA through extensive base pairing, leading to rapid mRNA degradation.'},
 {'entities_involved': ['miRNA', 'mRNA'],
  'interaction_type': 'regulation',
  'interaction_details':

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap=0)

splits = text_splitter.split_text(page_content)

In [ ]:
len(splits)

5

In [ ]:
# Define a function to flatten a 2D list (matrix) into a 1D list (flat list).
def flatten(matrix):
    flat_list = []
    # Iterate through each row in the 2D list.
    for row in matrix:
        flat_list += row
    # Return the flattened list.
    return flat_list

In [ ]:
import time

# Define a function that adds a delay to a Completion API call
def delayed_completion(delay_in_seconds: float = 1, **kwargs):
    """Delay a completion by a specified amount of time."""

    # Sleep for the delay
    time.sleep(delay_in_seconds)

    # Call the Completion API and return the result
    return ChatOpenAI(**kwargs)

In [ ]:
# Calculate the delay based on your rate limit
rate_limit_per_minute = 3
delay = 60.0 / rate_limit_per_minute

In [ ]:
timed_model = delayed_completion(delay_in_seconds=delay, model = "gpt-3.5-turbo",
                                 temperature=0, openai_api_key=OPENAI_API_KEY)

In [ ]:
extraction_timed_model = timed_model.bind(
    functions=paper_extraction_function,
    function_call={"name":"Molecular_Interactions"}
)

In [ ]:
extraction_timed_chain = prompt | extraction_timed_model | JsonKeyOutputFunctionsParser(key_name="interactions")

In [ ]:
from langchain.schema.runnable import RunnableLambda

In [ ]:
# Create a RunnableLambda which is a function that can be used in a chain.
# It takes a text input, splits it using the text splitter, and wraps each chunk in a dictionary.

prep = RunnableLambda(
    lambda x: [{"input": doc} for doc in text_splitter.split_text(x)]
)

In [ ]:
chain = prep | extraction_timed_chain.map() | flatten

In [ ]:
page_content = doc[1481:20000]
# print(page_content)

In [ ]:
results = chain.invoke(page_content)

In [ ]:
results

[{'entities_involved': ['miRNA', 'DNA'],
  'interaction_type': 'regulation',
  'interaction_details': 'miRNA regulates more than 60% of all human protein-coding genes at the post-transcriptional level.'},
 {'entities_involved': ['miRNA', 'mRNA'],
  'interaction_type': 'regulation',
  'interaction_details': 'miRNA enhances or represses the translation of mRNA at the post-transcriptional level.'},
 {'entities_involved': ['miRNA', 'messenger RNA (mRNA)'],
  'interaction_type': 'binding',
  'interaction_details': 'miRNA binds to mRNA to synergistically repress translation'},
 {'entities_involved': ['miRNA', 'mRNA'],
  'interaction_type': 'binding',
  'interaction_details': 'miRNA binds to mRNA with a short target sequence that may be common between different groups of miRNAs'},
 {'entities_involved': ['miRNA', 'mRNA'],
  'interaction_type': 'degradation',
  'interaction_details': 'miRNA triggers mRNA degradation pathway to downregulate protein production'},
 {'entities_involved': ['miRNA',

In [ ]:
import json


def convert_to_cx2(extracted_data):
    # Initialize the CX2 format structure
    cx2_network = {
        "nodes": [],
        "edges": [],
        "networkAttributes": [],
        "nodeAttributes": [],
        "edgeAttributes": []
    }

    # Define node and edge ids
    node_id = 1
    edge_id = 1

    # Maps to keep track of nodes and ensure they're unique
    nodes_map = {}

    # Process each interaction
    for interaction in extracted_data:
        for entity in interaction['entities_involved']:
            if entity not in nodes_map:
                nodes_map[entity] = node_id
                cx2_network['nodes'].append({"@id": node_id, "n": entity})
                node_id += 1

        source_node_id = nodes_map[interaction['entities_involved'][0]]
        target_node_id = nodes_map[interaction['entities_involved'][1]]

        # Add edge based on the interaction
        cx2_network['edges'].append({"@id": edge_id, "s": source_node_id, "t": target_node_id, "i": interaction['interaction_type']})
        edge_id += 1

        # Optionally, add edge attributes if interaction_details are present
        if interaction.get('interaction_details'):
            cx2_network['edgeAttributes'].append({
                "po": edge_id - 1,          #property_of edge attribute
                "n": "interaction_details",
                "v": interaction['interaction_details']
            })

    return cx2_network

In [ ]:
# Convert extracted data to CX2 format
cx2_network = convert_to_cx2(results)

# Convert CX2 data to JSON
cx2_json = json.dumps(cx2_network, indent=4)

In [ ]:
cx2_network

{'nodes': [{'@id': 1, 'n': 'miRNA'},
  {'@id': 2, 'n': 'mRNA'},
  {'@id': 3, 'n': 'AGO proteins'},
  {'@id': 4, 'n': 'HOXB8 mRNA'},
  {'@id': 5, 'n': 'TUSC2'},
  {'@id': 6, 'n': 'SERBP1'},
  {'@id': 7, 'n': 'Rtl1/Peg11'}],
 'edges': [{'@id': 1, 's': 1, 't': 2, 'i': 'regulation'},
  {'@id': 2, 's': 1, 't': 3, 'i': 'cleavage'},
  {'@id': 3, 's': 1, 't': 4, 'i': 'cleavage'},
  {'@id': 4, 's': 1, 't': 5, 'i': 'cleavage'},
  {'@id': 5, 's': 1, 't': 6, 'i': 'cleavage'},
  {'@id': 6, 's': 1, 't': 7, 'i': 'cleavage'}],
 'networkAttributes': [],
 'nodeAttributes': [],
 'edgeAttributes': [{'po': 1,
   'n': 'interaction_details',
   'v': "miRNA regulates mRNA translation by binding to the 3' UTR, coding sequence (CDS), or 5' UTR of mRNA. This interaction can lead to mRNA degradation, decreased ribosomal interaction, or upregulation of translation."},
  {'po': 2,
   'n': 'interaction_details',
   'v': 'miRNA can cleave target mRNAs in a site-specific manner, known as RNA interference (RNAi), utili